In [3]:
import pandas as pd
import numpy as np
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments
from transformers import TextClassificationPipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch

In [4]:
df = pd.read_csv("/content/Tweets_labelled.csv")
df.head()

,textID,text,selected_text,label
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [5]:
def clean_text(text):
    if isinstance(text, str):
        text = text.replace(r'http\S+|www.\S+', '')  # Remove URLs - remove case argument
        text = text.replace(r'@\w+', '')  # Remove mentions
        text = text.replace(r'#', '')  # Remove hashtags
        text = text.replace(r'[^\w\s]', '')  # Remove punctuation
        return text
    else:
        return ""  # Or handle non-string elements as needed

df['text'] = df['text'].apply(clean_text)  # Apply the cleaning function to each text

In [6]:
train_texts, test_texts, train_labels, test_labels = train_test_split(df['text'], df['label'], test_size=0.2)

In [7]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [8]:
train_texts = train_texts.tolist()
test_texts = test_texts.tolist()

In [9]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)


In [10]:
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        # Map string labels to integers if necessary
        label_mapping = {'negative': 0, 'neutral': 1, 'positive': 2}  # Adjust as needed
        item['labels'] = torch.tensor(label_mapping.get(self.labels[idx], self.labels[idx]))
        return item

    def __len__(self):
        return len(self.labels)

In [11]:
train_dataset = SentimentDataset(train_encodings, train_labels.tolist())
test_dataset = SentimentDataset(test_encodings, test_labels.tolist())


In [12]:
from transformers import DistilBertForSequenceClassification

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=3)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=11,              # total number of training epochs
    per_device_train_batch_size=10,  # batch size for training
    per_device_eval_batch_size=10,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    learning_rate=2e-5,              # Learning rate
    evaluation_strategy="epoch",     # evaluate at the end of each epoch
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset            # evaluation dataset
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.581700,0.522471
2,0.568200,0.554300
3,0.439500,0.686553
4,0.193100,1.004147
5,0.229500,1.143793
6,0.155300,1.400121
7,0.027600,1.466330
8,0.046600,1.648146
9,0.001300,1.713803
10,0.002300,1.741295


TrainOutput(global_step=24189, training_loss=0.20386379299647686, metrics={'train_runtime': 3083.2654, 'train_samples_per_second': 78.431, 'train_steps_per_second': 7.845, 'total_flos': 6882383640401280.0, 'train_loss': 0.20386379299647686, 'epoch': 11.0})

In [14]:
predictions = trainer.predict(test_dataset)
preds = np.argmax(predictions.predictions, axis=-1)


In [15]:
# Convert test_labels to integers using a mapping
label_mapping = {'negative': 0, 'neutral': 1, 'positive': 2}
test_labels = test_labels.map(label_mapping)

# Convert preds to integers if they are strings
if isinstance(preds[0], str):
    preds = preds.astype(int)

In [17]:
accuracy = accuracy_score(test_labels, preds)
precision, recall, f1, _ = precision_recall_fscore_support(test_labels, preds, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


Accuracy: 0.7820629434236857
Precision: 0.7829769881858407
Recall: 0.7820629434236857
F1 Score: 0.7823272107971548


In [18]:
nlp_pipeline = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [21]:
result = nlp_pipeline("Hy I am a good Soldier")

In [22]:
# prompt: use the above result and print wether the text is positive negative or nutral based on the score.
# the label that is closer to 1 is the sentiment
# label_0 for negative
# label_1 is for nutral
# label_2 is for positive in the results

def get_sentiment(result):
  scores = result[0]
  max_score = 0
  sentiment = ""
  for score in scores:
    if score['score'] > max_score:
      max_score = score['score']
      sentiment = score['label']

  if sentiment == 'LABEL_0':
    return "negative"
  elif sentiment == 'LABEL_1':
    return "neutral"
  elif sentiment == 'LABEL_2':
    return "positive"

print(get_sentiment(result))


positive


In [23]:
# prompt: pickle my trained model

import pickle

# Save the model
filename = 'trained_model.pkl'
pickle.dump(nlp_pipeline, open(filename, 'wb'))

# Load the model
loaded_model = pickle.load(open(filename, 'rb'))
